# OBRÓBKA DANYCH DLA MODELI ML

## 1) Obsługa brakujących danych

Większość bibliotek ML (włącznie z scikit-learn) zwraca błąd przy próbie budowy modeli na danych, w których występują braki (NA). Poniżej poruszone zostaną 3 podejścia stosowane do pracy z brakującymi danymi.


### Rzucenie okiem na dane

Pierwszą czynnością pracy z danymi jest ich wczytanie i wstępna (wizualna) analiza:
- dataset.head()
- dataset.describe()

Warto również zapoznać się z **dokumentacją** zbioru danych, jeśli korzystamy ze zbioru utworzonego przez kogoś innego (np. z kaggle.com).

### Określenie liczby brakujących danych (NaN, None)

Dobrą praktyką jest sprawdzenie czy nasz zbiór danych posiada jakieś braki:

<img src="Images/img_22.jpg">

Już po pierwszych 10 kolumnach widać, że mamy sporo brakujących wartości. Aby uchwycić skalę problemu najlepiej jest określić **procent** brakujących wartości w naszym zbiorze danych.

<img src="Images/img_23.jpg">

Widać wyraźnie, że blisko 1/4 komórek jest pusta (NaN lub None).

### Określenie skąd się biorą braki danych

_"Look at your data and try to figure out why it is the way it is and how that will affect your analysis"_

Aby sprawnie radzić sobie z brakami danych potrzebne jest doświadczenie (i intuicja!). Dobrym pytaniem, które warto sobie postawić próbując znaleźć odpowiedź na pytanie skąd biorą się brakujące wartości jest: 

**Czy konkretny brak danych wynika z faktu, że dana wartość nie została zarejestrowana, czy ponieważ w ogóle nie istnieje?**

Jeśli mamy do czynienia z brakiem, który wynika z faktu, że dana wartość nie istnieje (np. wzrost najstarszego dziecka kogoś, kto w ogóle nie ma dzieci), to nie ma sensu szukać sposobu na uzupełnienie takiego braku - najlepiej pozostawić wartość NaN.

Jeśli jednak brak wynika z tego, że dana wartość nie została zarejestrowana (ale prawdopodobnie może istnieć), wtedy można pokusić się o "przewidzenie" brakującej wartości w oparciu o istniejące wartości danej kolumny (cechy) - proces ten nosi nazwę **imputacji (ang. imputation)**.

Przykładowo, dla pierwszych 10 kolumn jak wyżej, kolumna _time_ określa liczbę sekund jakie pozostały do końca meczu w momencie wykonania akcji. Wynika stąd, że wszystkie braki w kolumnie _time_ wynikają z tego, że po prostu ich nie zarejestrowano! Stąd, sensowne wydaje się podjąć próbę ich zastąpienia zamiast pozostawić jako NA.

Z drugiej strony kolumna _PenalizedTeam_ również zawiera wiele braków, ale w tym przypadku biorą się one z faktu, że po prostu żadna z drużym nie została w danej akcji ukarana (nie było przewinienia), dlatego szukanie "na siłę" wartości (drużyny) do wstawienia jest bez sensu i należy pozostawić NA lub ewentualnie wstawić coś a'la "brak".

**WNIOSEK:**

Warto skupić się na każdej kolumnie z osobna i dość szczegółowo przeanalizować w jaki sposób najlepiej poradzić sobie z brakującymi wartościami.

### Podejście 1: Usunięcie wierszy lub kolumn z brakującymi danymi

Jedną z najszybszych (ale niekoniecznie najlepszych) metod radzenia sobie z brakami danych jest usunięcie wierszy (lub nawet kolumn, jeśli braków jest ponad 50% w danej kolumnie) zawierających braki.

<img src="Images/img_1.jpg">

Niestety, poprzez usuwanie całych kolumn znacznie zmniejsza się nasz zbiór danych pod kątem wyboru cech dla modelu (mamy mniej kolumn do wyboru przez co model może mieć później słabą jakość).

<img src="Images/img_24.jpg">

Jak widać, użycie metody "dropna" może spowodować usunięcie wszystkich wierszy, jeśli każdy z nich zawierał przynajmniej jedną brakującą wartość. W takiej sytuacji lepszym rozwiązaniem może być usunięcie kolumn, które posiadają braki:

<img src="Images/img_25.jpg">

<img src="Images/img_26.jpg">

Metoda polegająca na usunięciu kolumn również nie jest najlepsza, bo doprowadziła w powyższym przykładzie do ponad 2-krotnego zmniejszenia naszego zbioru danych...

**WSKAZÓWKA:**

Metoda całkowitego usuwania brakujących wartości nie jest rekomendowana w pracy przy ważnych komercyjnych projektach!

### Podejście 2: Imputacja - przypisanie nowych wartości

Lepsze rozwiązanie niż usuwanie całych kolumn. Polega na zastąpieniu brakujących wartości np. średnią wartością dla danej kolumny czy też wartością najczęściej występującą w danej kolumnie.

<img src="Images/img_2.jpg">

Wstawienie nowej wartości nie zawsze jest pożądane, ale zazwyczaj prowadzi do uzyskania modeli o wyższej jakości niż w przypadku usuwania całych kolumn.

(Poniższy przykład celowo dotyczy wycinka całego zbioru danych, aby pokazać o co chodzi.)

<img src="Images/img_27.jpg">

Używając modułu Pandas mamy możliwość skorzystania z metody _fillna(n)_ , która zastępuje wszystkie braki danych NAN występujące w DataFramie wskazaną wartością _n_ .

<img src="Images/img_28.jpg">

Innym, nieco sprytniejszym, sposobem jest zastąpienie braku wartością, która występuje bezpośrednio po nim (po braku) w tej samej kolumnie. Takie działanie ma sens w przypadku zbiorów, w których obserwacje mają jakiś logiczny porządek.

<img src="Images/img_29.jpg">

W przypadku biblioteki Scikit-Learn możliwe jest zastosowanie metody 'SimpleImputer', która zastąpi braki danych (NaN) w danej kolumnie np. wartością średnią.

### Podejście 3: Imputacja + INFO - przypisanie nowych wartości oraz uwzgl. info o tym w nowej kolumnie

W tym podejściu zastępujemy braki danych, podobnie jak to miało miejsce w punkcie 2, ale oprócz tego dodajemy nową kolumnę  z informacją o tym czy dane w danym wierszu zostały zastąpione nową wartością, czy nie.

<img src="Images/img_3.jpg">

W pewnych przypadkach, model posiadając wiedzę nt tego, które wartości zostały zastąpione może uzyskiwać lepsze wyniki. W innych - nie zmieni to kompletnie niczego. Przykład:

## 2) Obsługa danych liczbowych

(**Więcej**: książka, rozdz. 4)

Dane ilościowe pozwalają na pomiar wielkości dowolnych elementów: klasy, miesięcznej sprzedaży, pensji czy ocen uczniów. Naturalnym sposobem na przedstawienie tego pomiaru jest użycie liczb, np. 29 uczniów lub sprzedaż w wysokości 529 392 zł. Bardzo ważne jest zatem przekształcenie surowych danych liczbowych na cechy wykorzystywane później w algorytmach ML. 

Większość algorytmów uczenia maszynowego niestety słabo sobie radzi z atrybutami numerycznymi znajdującymi się w różnych zakresach skali (przykład: całkowita liczba pomieszczeń mieści się w zakresie od 6 do 39 320, z kolei wartości mediany dochodów to zakres zaledwie od 0 do 15). Konieczne jest zatem przeskalowanie cech, **polegające na modyfikacji danych w taki sposób, że mieszczą się one w określonym zakresie**, najczęściej od 0 do 1 lub od -1 do 1. 

Skalowanie jest jednym z najważniejszych przekształceń dokonywanych na danych liczbowych. Stosuje się je zazwyczaj, gdy używamy algorytmów opartych na miarach odległości między punktami danych, np. **SVM (Support Vector Machines)** lub **K-NN (K-Nearest Neighbours)**. W tych algorytmach zmiana o "1" w dowolnej funkcji numerycznej ma taką samą wagę.

Przykładowo, nasz zbiór danych może zawierać ceny niektórych produktów w jenach i dolarach. Jeden dolar to około 100 jenów, ale jeśli nie przeskalujemy cen w naszym zbiorze, to algorytmy takie jak SVM czy KNN potraktują różnicę w cenie 1 jena tak samo ważną, jak różnicę 1 dolara!!! A co jeśli mamy wzrost i wagę? Tutaj akurat nie da się jasno określić ile kg powinno równać się jednemu centymetrowi, dlatego skalowanie nie ma zastosowania.

Skalowanie umożliwa zatem porównywanie różnych zmiennych "na równych zasadach".

Wyróżnia się następujące metody skalowania:

### Skalowanie min - max

**Skalowanie min - max polega na modyfikacji danych w taki sposób, że mieszczą się one w określonym zakresie**, 

Istnieje wiele technik skalowania, a jedną z najprostszych jest tzw. **skalowanie min - max**. W trakcie tej operacji wartości minimalna i maksymalna cechy są używane do przeskalowania wartości w przedziale. Dokonujemy tego, odejmując od danej wartości wartość minimalną i dzieląc otrzymany wynik przez różnicę wartości maksymalnej i minimalnej. 

<img src="Images/img_77.jpg">

W module Scikit-Learn służy do tego funkcja transformująca *MinMaxScaler*. 

Zawiera ona hiperparametr *feature_range*, pozwalający zmieniać zakres skali, jeśli z jakiegoś powodu nie odpowiada nam domyślny zakres 0 – 1.

**Algorytmy ML wymagające skalowania min-max:**

- KNN
- regresja liniowa
- sieci neuronowe

**UWAGA!** Skalowanie min - max jest bardzo wrażliwe na wartości odstające (outliers).

(Poniżej przykład skalowania min-max, ale nie z modułu Scikit-Learn a z mlxtend.preprocessing, ale zasada działania taka sama).

<img src="Images/img_31.jpg">

Zauważmy, że _kształt_ danych na powyższym przykładzie nie zmienił się. Zmianie uległ natomiast zakres wartości (oś pozioma) z 0-8 na 0-1.

**Więcej + przykład z Scikit-Learn**: książka, str 73.

### Skalowanie robust

*RobustScaler* działa podobnie do skalera Min-Max, ale zamiast wartości min i max używa rozstępu międzykwartylnego. A zatem używa mniej danych do skalowania, więc jest bardziej odpowiedni dla sytuacji, gdy dane są odstające.


### Normalizacja

Terminy skalowania min-max i normalizacji często są (błędnie) stosowane zamiennie. Dzieje się tak, ponieważ oba procesy są bardzo podobne. Zarówno jeden, jak i drugi polega na transformacji danych liczbowych (zmiennych numerycznych) w taki sposób, aby dane te po przekształceniu miały pewne, określone (i pomocne z punktu widzenia modelu) właściwości. 

Jednym z przykładów normalizacji jest takie przekształcenie danych, że ich suma wynosi 1 (utrzymujemy tzw. jednostkę normatywną). Takie przeskalowanie jest często stosowane w przypadku występowania wielu odpowiedników cech, np. podczas klasyfikacji tekstu, gdy każde słowo lub co n-te słowo jest cechą. 

W bibliotece Scikit-Learn do tego typu przekształceń służy klasa *Normalizer*.

**Algorytmy ML wymagające normalizacji z użyciem klasy *Normalizer*:**

Takie przetwarzanie może być użyteczne w przypadku rzadkich zestawów danych (wiele zer) z atrybutami o różnej skali w przypadku korzystania z algorytmów ważących wartości wejściowe, tj.

- sieci neuronowe 
- algorytmy wykorzystujące pomiary odległości, np. KNN.

**Więcej + przykład**: książka, str 76.

Innym przykładem transformacji rozumianym jako "normalizacja", jest **zmianie kształtu rozkładu (shape of the distribution) danych**. Jej celem jest zmiana obserwacji w ten sposób, aby dało się je opisać za pomocą **rozkładu normalnego** (mniej więcej). Jedną z metod pozwalających uzyskać rozkład normalny jest metoda **Box'a-Cox'a**:

<img src="Images/img_30a.jpg">
<img src="Images/img_32.jpg">

Zauważmy, że _kształt_ danych na powyższym przykładzie uległ zmianie. Przed normalizacją był niemal w kształcie litery "L", zaś po normalizacji przypomina rozkład normalny (krzywa dzwonowa).

Normalizacji danych do rozkładu normalnego można dokonać również z wykorzystaniem pakietu *NumPy* i **logarytmu naturalnego** ( *all_data['norm_fare'] = np.log(all_data.Fare* )

Ogólnie rzecz biorąc, normalizację do rozkładu normalnego stosuje się, gdy zamierzamy używać modelu uczenia maszynowego lub techniki statystycznej, która zakłada, że dane posiadają rozkład normalny. 

**Algorytmy ML wymagające normalizacji do rozkładu normalnego:**

- regresja liniowa
- naiwny klasyfikator Bayesowski
- metody zawierające "Gaussian" w nazwie (w ciemno)

oraz

- analiza wariancji (ANOVA)
- liniowa analiza dyskryminacyjna (LDA)


### Standaryzacja

Standaryzację stosuje się często w celu poprawy wyników klasyfikacji. Metoda ta polega na przesunięciu danych o ich wartość średnią w kierunku zera i późniejszym podzieleniu przez wartość odchylenia standardowego. Inaczej, jest to takie przekształcenie cechy, aby jej średnia wynosiła 0, a odchylenie standardowe 1 (zapewnia to mniej więcej równy rozkład cech). Przekształcona cecha pokazuje o ile odchylenie standardowe wartości początkowej różni się od wartości średniej cechy (jest to tzw. **wskaźnik z-score**)

Obliczanie metodą standaryzacji polegaja na tym, że od danej wartości odejmujemy średnią, a następnie wynik dzielimy przez wariancję, dzięki czemu wynikowy rozkład ma wariancję jednostkową.

Standaryzacja jest bardzo często stosowaną alternatywą dla skalowania min-max i daje ona bardzo dobre wyniki, jesli zamierzamy korzystać z analizy głównych składowych (PCA). Ponadto, standaryzacja zakłada, że obserwacje **mają rozkład normalny (!)** (patrz: normalizacja).

W przeciwieństwie do skalowania min-max, standaryzacja nie ogranicza skalowanych wartości do określonego zakresu, co może stanowić problem dla niektórych algorytmów (np. sieci neuronowe często oczekują wartości wejściowych mieszczących się w zakresie 0-1). Z drugiej strony standaryzacja jest mniej wrażliwa na elementy odstające

**Algorytmy ML wymagające standaryzacji:**

- SVM (Support Vector Machines)
- regresja liniowa
- regresja logistyczna
- Ogólnie: algorytmy mające lepszą wydajność, jeśli ich cechy posiadają rozkład normalny (Gaussa)

W Scikit-Learn standaryzacja występuje jako klasa *StandardScaler*.

**Więcej + przykład**: książka, str 74.


### Dyskretyzacja (binaryzacja)

Przedstawione powyżej metody koncentrowały się na skalowaniu danych. W odróżnieniu od nich *dyskretyzacja* polega na zamianie wartości atrybutów na "grupy" (w postaci liczbowej) w zależności od wartości ustalonego progu. I tak, chcąc podzielić dane mamy do dyspozycji dwie techniki:

1. Podział cechy liczbowej na podstawie pojedynczej wartości progowej (binaryzacja):

Wszystkie wartości większe od progu zamieniane są na 1, a pozostałe - na 0. Pozwala to interpretować obserwacje, jako realizacje schematu Bernoulliego, gdzie 1 oznacza sukces, a 0 porażkę.

2. Podział cechy liczbowej na podstawie wielu wartości progowych 

Otrzymamy więcej grup niż dwie: 0, 1, 2, 3, 4...

Dyskretyzacja jest przydatna, gdy istnieje przekonanie, że cecha liczbowa powinna zachowywać się bardziej jak cecha kategoryczna. Przykładowo możemy zakładać niewielką różnicę w nawykach wydawania pieniędzy przez osoby w wieku 19 i 20 lat oraz znacznej przez osoby w wieku 20 i 21 lat (w USA alkohol może kupić dopiero osoba, która ukończyła 21 lat) - w takim przypadku użyteczne może być przeprowadzenie podziału obserwacji na przedstawiające osoby, które mogą i nie mogą spożywać alkoholu (binaryzacja). W innych sytuacjach użyteczna może się okazać dyskretyzacja danych na trzy lub więcej kategorii.

W Scikit-Learn standaryzacja występuje jako klasa *Binarizer*.

**Więcej + przykład**: książka, str 84.



## Outliers (elementy odstające)

https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame/23200666#23200666


#### Wykrywanie elementów odstających:

Niestety nie ma jednej najlepszej techniki przeznaczonej do wykrywania elementów odstających.

Chcąc wykryć elementy odstające w poszczególnych cechach można skorzystać z **rozstępu ćwiartkowego (IQR)**. Podaje on różnicę między pierwszą a trzecią ćwiartką zbioru danych i można potraktować go jako rozrzut danych, w którym elementy odstające to obserwacje znacznie oddalone od głównego miejsca koncentracji danych. Za element odstający jest zwykle uznawana każda wartość 1.5 IQR razy mniejsza niż pierwsza ćwiartka lub 1.5 IQR razy większa niż trzecia ćwiartka.

**Więcej + przykład**: książka, str 80.

#### Obsługa elementów odstających:

Podobnie, jak w przypadku wykrywania elementów odstających, także w zakresie ich obsługi nie obowiązuje żadna konkretna reguła.

**UWAGA!** Jeśli istnieją elementy odstające, wówczas nie powinno stosować się standaryzacji, która jest wrażliwa na el. odstające. Lepiej zastosować np. klasę *RobustScaler*.

**Więcej + przykład**: książka, str 82.

## 4) Obsługa danych kategorycznych

**Więcej**: https://kiwidamien.github.io/encoding-categorical-variables.html

**Więcej**: książka, rozdz. 5

Bardzo często użyteczne jest mierzenie obiektów nie w kategoriach ich ilości, ale pewnej jakości. Związane z tym informacje są z reguły przedstawiane, jako obserwacje w oddzielnych kategoriach, takich jak np. płeć, kolor lub marka samochodu. Zmienne kategoryczne posiadają zatem organiczoną liczbę wartości. Ogólnie, zmienne kategoryczne są jasno określone i jest ich skończona ilość (jasno wskazują daną kategorię)

Jednak nie wszystkie dane kategoryczne są takie same. Zbiór kategorii bez powiązanej z nimi kolejności jest określany mianem kategorii **nominalnej**:

- niebieski, czerwony, zielony
- mężczyzna, kobieta
- banan, truskawka, jabłko

Natomiast zbiór kategorii zawierających pewną naturalną kolejność jest określany mianem kategorii **porządkowej**, np.:

- mało, średnio, dużo
- młody, stary
- pozytywny, neutralny, negatywny

W Pythonie, jeśli zechcemy użyć zmiennych kategorycznych do budowy modelu ML w ich podstawowej formie, to dostaniemy błąd, dlatego należy poddać je "preprocessingowi" i **zamienić je na wartości liczbowe**. 

Poniżej opis wybranych podejść pracy ze zmiennymi kategorycznymi.

### Całkowite usunięcie cech kategoryzujących:

Najłatwiejszym rozwiązaniem jest usunięcie zmiennych (kolumn) zawierających wartości kategoryczne. Działa tylko, jeśli kolumny nie zawierają informacji istotnych z punktu widzenia modelu.

### Kodowanie nominalnych cech kategoryzujących:

**One-Hot Encoding / Dummying (kodowanie "gorącojedynkowe")**

Polega na utworzeniu nowego zestawu kolumn, który jasno określa czy dana wartość zmiennej kategorycznej występuje w zbiorze danych, czy nie (jest to tworzenie cechy binarnej dla każdej klasy oryginajnej cechy). Oznacza to, że każda klasa staje się samodzielną cechą binarną, gdzie 1 oznacza wystąpienie danej cechy, a 0 - brak.

<img src="Images/img_5.jpg">

One-Hot Encoding nie porządkuje kategorii, dlatego podejście to sprawdza się, gdy nie jest jasny porządek wartości zmiennej kategorycznej (np. "Red" nie jest ani większy ani zmiejszy od "Yellow")

Podejście One-Hot Encoding nie działa dobrze, jeśli zmienna kategoryczna przyjmuje zbyt wiele unikalnych wartości (zakłada się, że nie używa się tego podejścia jeśli unikalnych wartości > 15. 

W Scikit-Learn: *LabelBinarizer*, *MultilabelBinarizer* lub *OneHotEncoder*.

**Więcej**: książka, str 92

**Więcej**: notatnik *Intermediate Machine Learning*

### Kodowanie porządkowych cech kategoryzujących:

**Label Encoding**

Polega na przypisaniu etykietom wartości liczbowych (int). Jest to swego rodzaju porządkowanie etykiet, np. "Never" (0) < "Rarely" (1) < "Most days" (2) < "Every day" (3).

<img src="Images/img_4.jpg">

Powyższe podejście ma sens, ponieważ kategorie (etykiety) mają swój "ranking". Nie wszystkie zmienne kategoryczne mają jednak wartości, które da się uporządkować (inaczej, nie wszystkie zmienne kategoryczne działają, jak zmienne porządkowe!). W przypadku Drzew i Lasów można jednak zakładać, że kodowanie (jw) zmiennych porządkowych przyniesie dobry wynik.

WAŻNE!
Jeśli w kolumnie zbioru walidacyjnego występują dane kategoryczne, których wartości nie występują w tej samej kolumnie zbioru treningowego, to otrzymamy BŁĄD! Najprostszym rozwiązaniem w takiej sytuacji jest po prostu całkowite usunięcie problematycznej kolumny.

**Więcej**: książka, str 94

**Więcej**: notatnik *Intermediate Machine Learning*



### Inne metody kodowania cech kategoryzujących:

**Więcej**: notatnik *Feature_Engineering*

#### Count Encoding

Polega na zastąpieniu każdej z wartości kategorycznych liczbą jej wystąpień w zbiorze danych. Przykładowo, jeśli wartość "GB" pojawia się w zbiorze 10 razy, to każde wystąpienie "GB" zostanie zastąpione liczbą 10. 

Chcąc skorzystać z tej metody należy użyć biblioteki 'category_encoders' oraz metody 'CountEncoder', która działa z metodami '.fit' oraz '.transform' pochodzącymi z pakietu Scikit-Learn.

<img src="Images/img_10.jpg">

#### Target Encoding

Polega na zastąpieniu wartości kategorycznych średnią wartością obliczoną na podstawie poszczególnych wartości zmiennej przewidywanej (target) dla danej wartości zmiennej kategorycznej (inaczej, grupuje unikalne wartości zmiennej kategorycznej, oblicza dla każdej grupy średnią wartość targetu i wstawia w każdy wiersz należący do tej grupy).

Ta technika wykorzystuje zmienną docelową (target) do utworzenia nowej cechy (która ma zastąpić cechę "kategorczną"), dlatego wykorzystanie tej nowej cechy do treningu lub walidacji modelu byłoby w pewnym sensie **wyciekiem danych** (data leakage). Należy zatem zastosować "target encodings" jedynie dla zbioru treningowego.

Chcąc skorzystać z tej metody należy użyć biblioteki 'category_encoders' oraz metody 'TargetEncoder', która działa z metodami '.fit' oraz '.transform' pochodzącymi z pakietu Scikit-Learn.

<img src="Images/img_11.jpg">

#### CatBoost Encoding

Metoda ta jest podobna do "Target Encoding", ponieważ również opiera się na wartościach zmiennej przewidywanej (target), jednakże stosując "CatBoost Encoding" wartość prawdopodobieństwa jest obliczana jedynie na podstawie wierszy poprzedzających wiersz, dla którego chcemy obliczyć nową wartość zmiennej kategorycznej.

<img src="Images/img_12.jpg">